In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import xgboost as xgb

In [2]:
holiday = ['2017-01-01','2017-01-02','2017-01-27','2017-01-28','2017-01-29','2017-01-30','2017-01-31','2017-02-01','2017-02-02','2017-04-02','2017-04-03','2017-04-04','2017-04-29',
          '2017-04-30','2017-05-01','2017-05-28','2017-05-29','2017-05-30','2017-10-01','2017-10-02','2017-10-03','2017-10-04','2017-10-05','2017-10-06','2017-10-07','2017-10-08']

In [3]:
northesat = ['黑龙江省','吉林省','辽宁省']
huaeast = ['上海市','江苏省','浙江省','安徽省','福建省','江西省','山东省','台湾省']
huanorth = ['北京市','天津市','山西省','河北省','内蒙古自治区']
huacenter = ['湖北省','湖南省','河南省']
huasouth = ['广东省','广西壮族自治区','海南省','香港特别行政区','澳门特别行政区']
southwest = ['四川省','贵州省','云南省','重庆市','西藏自治区']
northwest = ['陕西省','甘肃省','青海省','宁夏回族自治区','新疆维吾尔自治区']

In [39]:
data = pd.read_csv('detail_weather all_loc.csv')

C:\Users\user\Anaconda2\envs\py35\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (38,39) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [40]:
oil = pd.read_csv('oil_fill.csv')
oil.drop('Unnamed: 0',axis=1,inplace=True)

In [41]:
data.drop(['起运地经度','起运地纬度','目的地经度','目的地纬度','start_formatted_address','start_district','start_city','start_township','desti_formatted_address','desti_district','desti_city','desti_township','district_x','city_x','province_x','start_url','start_wind_direct','start_wind_scale','district_y','city_y','province_y','desti_url','desti_wind_direct','desti_wind_scale'],axis=1,inplace=True)

In [42]:
calc_nan = data[data['一级品类'] == 1][data['订单类型'] == 0][data['交易类型'] == 1][data['车型'] == 0]

C:\Users\user\Anaconda2\envs\py35\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [43]:
data.loc[data['货值（吨/元）'].isnull(),'货值（吨/元）'] = calc_nan['货值（吨/元）'].median()

In [44]:
def calc_length(data):
    cate = data['车型'].unique().tolist()
    dd = pd.DataFrame()
    for i in cate:
        cate_data = data[data['车型'] == i]
        cate_data.loc[cate_data['车长'].isnull(),'车长'] = cate_data.loc[data['车长'].notnull()]['车长'].median()
        dd = dd.append(cate_data)
    return dd

In [45]:
data = calc_length(data)

C:\Users\user\Anaconda2\envs\py35\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [46]:
data = data.sort_values(by='Unnamed: 0',axis=0)

In [47]:
data.drop('Unnamed: 0',axis=1,inplace=True)

In [48]:
data = pd.merge(data,oil,on=['日期','start_province'],how='left')

In [49]:
data.drop(['start_weather','desti_max_temp','desti_min_temp','desti_weather'],axis=1,inplace=True)

In [50]:
data = data.join(pd.get_dummies(data['车型'],prefix='车型_'))

In [51]:
data.drop('车型',axis=1,inplace=True)

In [14]:
#data.to_csv('contest.csv')

In [30]:
data = pd.read_csv('contest.csv')

In [52]:
data.drop('详细品类',axis=1,inplace=True)

In [23]:
def date_transform(date):
    if date in holiday:
        x = 1
    else:
        x = 0
    return x

In [24]:
date = data[['日期']].applymap(date_transform)
date.rename(columns={'日期':'节日'},inplace=True)
data = data.join(date)
data.drop('日期',axis=1,inplace=True)

In [53]:
def season_transform(date):
    if date[5:7] == '01' or date[5:7] == '02' or date[5:7] == '03':
        x = 1
    elif date[5:7] == '04' or date[5:7] == '05' or date[5:7] == '06':
        x = 2
    elif date[5:7] == '07' or date[5:7] == '08' or date[5:7] == '09':
        x = 3
    elif date[5:7] == '10' or date[5:7] == '11' or date[5:7] == '12':
        x = 4
    return x

In [54]:
date = data[['日期']].applymap(season_transform)
date.rename(columns={'日期':'季节'},inplace=True)
data = data.join(date)
date_dummies = pd.get_dummies(data['季节'])
date_dummies.rename(columns={1:'春',2:'夏',3:'秋',4:'冬'},inplace=True)
data = data.join(date_dummies)
data.drop('季节',axis=1,inplace=True)

In [55]:
data.drop('日期',axis=1,inplace=True)

In [56]:
data = data.join(pd.get_dummies(data['一级品类']))
data.rename(columns={1:'品类1',2:'品类2',3:'品类3',4:'品类4'},inplace=True)

In [57]:
data.drop('一级品类',axis=1,inplace=True)

In [58]:
data.loc[data['desti_province'] == '[]','desti_province'] = '天津市'

In [59]:
def province_transform(province):
    if province in northesat:
        x = 1
    elif province in huaeast:
        x = 2
    elif province in huanorth:
        x = 3
    elif province in huacenter:
        x = 4
    elif province in huasouth:
        x = 5
    elif province in southwest:
        x = 6
    elif province in northwest:
        x = 7
    return x

In [60]:
start_district = data[['start_province']].applymap(province_transform)
start_district.rename(columns={'start_province':'start_district'},inplace=True)
data = data.join(start_district)

In [61]:
desti_district = data[['desti_province']].applymap(province_transform)
desti_district.rename(columns={'desti_province':'desti_district'},inplace=True)
data = data.join(desti_district)

In [62]:
start_district_dummies = pd.get_dummies(data['start_district'],prefix='start_district_')
desti_district_dummies = pd.get_dummies(data['desti_district'],prefix='desti_district_')

In [63]:
data = pd.concat([data,start_district_dummies,desti_district_dummies],axis=1)

In [64]:
data.drop(['start_province','desti_province','start_district','desti_district'],axis=1,inplace=True)

In [65]:
mean_temp = []
for i in range(len(data)):
    temp = (data['start_max_temp'][i] + data['start_min_temp'][i]) / 2
    mean_temp.append(temp)
mean_t = pd.DataFrame(mean_temp)

In [66]:
mean_t.rename(columns={0:'平均温度'},inplace=True)

In [67]:
data = data.join(mean_t)
data.drop(['start_max_temp','start_min_temp'],axis=1,inplace=True)

In [69]:
data.to_csv('train_features(season).csv')

In [26]:
data = pd.read_csv('final_features.csv')
data.drop('Unnamed: 0',axis=1,inplace=True)

In [68]:
std_oil_value = data[['oil_value']].apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))
std_oil_value.rename(columns={'oil_value':'标准化油价'},inplace=True)
data = data.join(std_oil_value)
std_value = data[['货值（吨/元）']].apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))
std_value.rename(columns={'货值（吨/元）':'标准化货值（吨/元）'},inplace=True)
data = data.join(std_value)
std_dist = data[['运距']].apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))
std_dist.rename(columns={'运距':'标准化运距'},inplace=True)
data = data.join(std_dist)
std_car_size = data[['车长']].apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))
std_car_size.rename(columns={'车长':'标准化车长'},inplace=True)
data = data.join(std_car_size)
std_temp = data[['平均温度']].apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))
std_temp.rename(columns={'平均温度':'标准化平均温度'},inplace=True)
data = data.join(std_temp)
data.drop(['oil_value','货值（吨/元）','运距','车长','平均温度'],axis=1,inplace=True)

In [21]:
y = data['运价（元/吨）']
data.drop('运价（元/吨）',axis=1,inplace=True)

In [23]:
x_train,x_test,y_train,y_test = train_test_split(data,y,train_size=0.8,random_state=1)
data_train = xgb.DMatrix(x_train, label=y_train)
data_test = xgb.DMatrix(x_test, label=y_test)
watch_list = [(data_test, 'eval'), (data_train, 'train')]

C:\Users\user\Anaconda2\envs\py35\lib\site-packages\sklearn\model_selection\_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [24]:
param = {'max_depth': 6, 'eta': 0.1, 'silent': 1, 'objective': 'reg:tweedie'}
bst = xgb.train(param, data_train, num_boost_round=500, evals=watch_list)
y_hat = bst.predict(data_test)
rmse = np.sqrt(np.average((y_hat - np.array(y_test))**2))

[0]	eval-tweedie-nloglik@1.5:205.764	train-tweedie-nloglik@1.5:206.237
[1]	eval-tweedie-nloglik@1.5:187.24	train-tweedie-nloglik@1.5:187.669
[2]	eval-tweedie-nloglik@1.5:170.582	train-tweedie-nloglik@1.5:170.969
[3]	eval-tweedie-nloglik@1.5:155.618	train-tweedie-nloglik@1.5:155.969
[4]	eval-tweedie-nloglik@1.5:142.197	train-tweedie-nloglik@1.5:142.514
[5]	eval-tweedie-nloglik@1.5:130.179	train-tweedie-nloglik@1.5:130.466
[6]	eval-tweedie-nloglik@1.5:119.439	train-tweedie-nloglik@1.5:119.698
[7]	eval-tweedie-nloglik@1.5:109.86	train-tweedie-nloglik@1.5:110.095
[8]	eval-tweedie-nloglik@1.5:101.342	train-tweedie-nloglik@1.5:101.554
[9]	eval-tweedie-nloglik@1.5:93.7869	train-tweedie-nloglik@1.5:93.9787
[10]	eval-tweedie-nloglik@1.5:87.1079	train-tweedie-nloglik@1.5:87.281
[11]	eval-tweedie-nloglik@1.5:81.2248	train-tweedie-nloglik@1.5:81.3808
[12]	eval-tweedie-nloglik@1.5:76.0642	train-tweedie-nloglik@1.5:76.2035
[13]	eval-tweedie-nloglik@1.5:71.5558	train-tweedie-nloglik@1.5:71.6792
[14]	

In [25]:
rmse

107.89069803898158

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 213645 entries, 0 to 213644
Data columns (total 48 columns):
订单类型                 213645 non-null int64
交易类型                 213645 non-null int64
运价（元/吨）              213645 non-null int64
车型__0                213645 non-null int64
车型__2                213645 non-null int64
车型__3                213645 non-null int64
车型__4                213645 non-null int64
车型__5                213645 non-null int64
车型__6                213645 non-null int64
车型__7                213645 non-null int64
车型__8                213645 non-null int64
车型__9                213645 non-null int64
车型__10               213645 non-null int64
车型__11               213645 non-null int64
车型__12               213645 non-null int64
车型__13               213645 non-null int64
车型__14               213645 non-null int64
车型__15               213645 non-null int64
车型__16               213645 non-null int64
车型__17               213645 non-null int64
车型__19               213645 n